In [15]:
import pandas as pd

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
import os

# Set Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = "aadibery2"
os.environ['KAGGLE_KEY'] = "b8024084f49314e4474eecb7851f917c"

In [9]:
print(os.getcwd())  # Prints the current working directory


c:\Users\aadib\OneDrive\Documents\GitHub\hackathonTEAM\ai-ml-analy


In [10]:
import zipfile

zip_file_path = "daily-transactions-dataset.zip"  # Path to the downloaded zip file
extract_to_path = "./data"  # Directory to extract the contents

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Dataset extracted to {extract_to_path}")

Dataset extracted to ./data


In [12]:
# print the stuff in .data
extracted_folder = './data'  # Update with your directory path if different
print(os.listdir(extracted_folder))

['Daily Household Transactions.csv']


In [18]:
file_path = "./data/Daily Household Transactions.csv"
data = pd.read_csv(file_path)

In [20]:
print(data.head())

with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
    display(data)

                  Date                   Mode  ... Income/Expense Currency
0  20/09/2018 12:04:08                   Cash  ...        Expense      INR
1  20/09/2018 12:03:15                   Cash  ...        Expense      INR
2           19/09/2018  Saving Bank account 1  ...        Expense      INR
3  17/09/2018 23:41:17  Saving Bank account 1  ...        Expense      INR
4  16/09/2018 17:15:08                   Cash  ...        Expense      INR

[5 rows x 8 columns]


,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency
0,20/09/2018 12:04:08,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR
1,20/09/2018 12:03:15,Cash,Food,snacks,Idli medu Vada mix 2 plates,60.0,Expense,INR
2,19/09/2018,Saving Bank account 1,subscription,Netflix,1 month subscription,199.0,Expense,INR
3,17/09/2018 23:41:17,Saving Bank account 1,subscription,Mobile Service Provider,Data booster pack,19.0,Expense,INR
4,16/09/2018 17:15:08,Cash,Festivals,Ganesh Pujan,Ganesh idol,251.0,Expense,INR
...,...,...,...,...,...,...,...,...
2456,1/1/2015,Cash,Transportation,NaN,share jeep - Place T base to top,20.0,Expense,INR
2457,1/1/2015,Cash,Transportation,NaN,share auto - Place H to Place T base,20.0,Expense,INR
2458,1/1/2015,Cash,Transportation,NaN,bus - brc to Place H,30.0,Expense,INR
2459,1/1/2015,Cash,Food,NaN,tea,10.0,Expense,INR


In [21]:
currency_rate = 0.012

data['amount-usd'] = data['Amount'] * currency_rate

In [22]:
data.head()

,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency,amount-usd
0,20/09/2018 12:04:08,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR,0.360
1,20/09/2018 12:03:15,Cash,Food,snacks,Idli medu Vada mix 2 plates,60.0,Expense,INR,0.720
2,19/09/2018,Saving Bank account 1,subscription,Netflix,1 month subscription,199.0,Expense,INR,2.388
3,17/09/2018 23:41:17,Saving Bank account 1,subscription,Mobile Service Provider,Data booster pack,19.0,Expense,INR,0.228
4,16/09/2018 17:15:08,Cash,Festivals,Ganesh Pujan,Ganesh idol,251.0,Expense,INR,3.012


In [42]:
# we now want to create an additional csv file that just has the category and note/description with it
cleaned_data = data[['Category', 'Note']]
print(cleaned_data.head())

         Category                         Note
0  Transportation         2 Place 5 to Place 0
1            Food  Idli medu Vada mix 2 plates
2    subscription         1 month subscription
3    subscription            Data booster pack
4       Festivals                  Ganesh idol


In [43]:
# Save the cleaned data to a new CSV file
cleaned_data.to_csv('cleaned_dataset.csv', index=False)

# Print confirmation
print("Cleaned data saved to 'cleaned_dataset.csv'")


Cleaned data saved to 'cleaned_dataset.csv'


In [45]:
# training the model with our trained data
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFaceModel

# Load the dataset from S3
bucket = 'userdatas3'
input_data = 's3://{}/cleaned_dataset.csv'.format(bucket)

data = pd.read_csv(input_data)

# Preprocess data: Prepare labels and descriptions for model training
labels = data['Category']
descriptions = data['Note']

# Tokenize the descriptions (using Hugging Face tokenizer, for example)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(descriptions.tolist(), truncation=True, padding=True, max_length=128)

# Define input features and labels
X_train = encodings['input_ids']
y_train = labels.tolist()

# Save preprocessed data to S3 for training
train_data = pd.DataFrame({'text': descriptions, 'label': labels})
train_data.to_csv('train_data.csv', index=False)

# Upload preprocessed training data to S3
train_data_s3 = 's3://{}//train_data.csv'.format(bucket)
train_data.to_csv(train_data_s3, index=False)

# Now that we have our data ready, we can proceed to set up a training job


ModuleNotFoundError: No module named 'sagemaker'